En este apartado se trabaja los archivos csv d_games, d_item_user, d_reviuws
en el que se realizan transformaciones y se extraen campos especificos para la creacion de nuevos DataSet:
- df_gastos
- df_genre_rankin
- df_genre
- df_developer

Los cuales son de utilidad para el desarrollo de las funciones de consultas

In [1]:

# Importamos librerias
import pandas as pd
import ast

In [3]:
# Se cargan los archivos csv
df_game = pd.read_csv('d_games.csv')
df_item = pd.read_csv('d_item_user.csv')
df_reviu = pd.read_csv('d_reviuws.csv')

1. FUNCION USERDATA

Desarrollo y creacion de la funcion

Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [2]:
#Se nesecita crear un dataframe con los sgte campos de df_item
d_gastos = df_item[['items_count', 'user_id', 'item_id']]


NameError: name 'df_item' is not defined

In [9]:
# Se crea el Dataframe con los precios, se eliminan duplicados y # Se renombra 'id' por 'item_id'

d_precio = df_game[['price', 'id']]
d_precio = d_precio.drop_duplicates(subset='id', keep='first')
# Se renombra 'id' por 'item_id' para unir mas adelante
d_precio = d_precio.rename(columns={'id':'item_id'})


In [10]:
# Se realiza la unios de los dos daataset anteriores atraves del campo 'item_id'
d_gastos = d_gastos.merge(d_precio, on='item_id', how='left')


In [11]:
#Se elimina la columna 'item_id' y se agrupa por usuario para sumar la cantidad gastada por usuario.
d_gastos = d_gastos.drop('item_id', axis=1)




In [13]:
d_gastos.head()

,items_count,user_id,price
0,277,76561197970982479,9.99
1,277,76561197970982479,4.99
2,277,76561197970982479,4.99
3,277,76561197970982479,4.99
4,277,76561197970982479,4.99


In [12]:
d_gastos_user = d_gastos.groupby('user_id')['price'].sum().reset_index()
d_gastos_user.head()


,user_id,price
0,--000--,397.78
1,--ace--,166.82
2,--ionex--,99.93
3,-2SV-vuLB-Kg,427.50
4,-404PageNotFound-,1509.32


In [14]:
#Cantidad de item x usuario
d_cant_item = d_gastos[['items_count', 'user_id']]
d_cant_item = d_cant_item.drop_duplicates(subset='user_id', keep='first')

d_cant_item.head()


,items_count,user_id
0,277,76561197970982479
277,888,js41637
1165,137,evcentric
1302,328,Riot-Punch
1630,541,doctr


In [15]:
# Se concatenan los Dataframe cantidad de item y gastos
d_gastos = d_cant_item.merge(d_gastos_user, on='user_id', how='right')
d_gastos.head()


,items_count,user_id,price
0,58,--000--,397.78
1,44,--ace--,166.82
2,23,--ionex--,99.93
3,68,-2SV-vuLB-Kg,427.50
4,149,-404PageNotFound-,1509.32


In [16]:
# Exportamos el Dataframe gastos
d_gastos.to_csv('df_gastos.csv')

In [17]:
# Creamos la funcion 'userdata'
def userdata(user_id):
    
    usuario = df_reviu[df_reviu['user_id'] == user_id]
    cantidad_dinero = d_gastos[d_gastos['user_id']== user_id]['price'].iloc[0]
    count_items = d_gastos[d_gastos['user_id']== user_id]['items_count'].iloc[0]
    
    total_recomendaciones = usuario['recommend'].sum()
    total_reviews = len(df_reviu['user_id'].unique())
    porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100

    return {
        'cantidad_dinero': cantidad_dinero,
        'porcentaje_recomendacion': round(porcentaje_recomendaciones, 2),
        'total_items': count_items.astype(int)
    }

In [18]:
userdata('EchoXSilence')

{'cantidad_dinero': 189.84, 'porcentaje_recomendacion': 0.0, 'total_items': 23}

2. FUNCION COUNTREVIEW

Creacion de la funcion

Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

In [19]:
def countreviews(fi, ff):
    
    userfechas = df_reviu[(df_reviu['date'] >= fi) & (df_reviu['date'] <= ff)]
    total_user = userfechas['user_id'].nunique()
    recomendacion = len(userfechas)
    total_recomendaciones = userfechas['recommend'].sum()
    porcentaje = (total_recomendaciones / recomendacion) * 100
    
    return {
        'total_review': total_user,
        'porcentaje': round(porcentaje,2)
    }

In [20]:
countreviews('2011-10-05', '2012-12-10')


{'total_review': 807, 'porcentaje': 98.63}

3. FUNCION GENRE

Desarrollo y Creacion de la Funcion

Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

In [17]:
# Se concatenan estos dos Dataframe para extraer los camposcrequeridos para la funcion 
df_joined = pd.merge(df_item, df_game, left_on='item_id', right_on='id')


In [24]:
df_joined.shape

(4167341, 18)

In [26]:
# Se extraen en nuevo Dataframe los sgte genres, user_id, user_url, playtime
df_genre = subset_df[['genres', 'user_id', 'user_url','playtime_forever']]


In [27]:
# Del campo genres utilizando ast.literal_eval para las listas de generos
df_genre['genres'] = df_genre['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)



C:\Users\angel\AppData\Local\Temp\ipykernel_14764\3790331642.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre['genres'] = df_genre['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [30]:
# Se crea cada genero como un nuevo registro
df_genre = df_genre.explode('genres')
df_genre.head()


,genres,user_id,user_url,playtime_forever
0,Action,76561197970982479,http://steamcommunity.com/profiles/76561197970...,6
1,Action,js41637,http://steamcommunity.com/id/js41637,0
2,Action,Riot-Punch,http://steamcommunity.com/id/Riot-Punch,0
3,Action,doctr,http://steamcommunity.com/id/doctr,93
4,Action,corrupted_soul,http://steamcommunity.com/id/corrupted_soul,108


In [82]:
# Se obtiene la suma de playtime para cada genero
df_genre_t = df_genre.groupby('genres').agg({'playtime_forever': 'sum'}).reset_index()


In [83]:
# Se ordenan de mayor a menor de platime cada registro
df_genre_t = df_genre_t.sort_values(by='playtime_forever', ascending=False).reset_index(drop=True)


In [84]:
# Se establece un campo ranking de playtime 
df_genre_t['ranking'] = range(1, len(df_genre_t)+1)
df_genre_t.head()


,genres,playtime_forever,ranking
0,Action,3070177095,1
1,Indie,1475182802,2
2,RPG,1027647746,3
3,Adventure,893956775,4
4,Simulation,855259647,5


In [90]:
# Se crea el archivo csv de ranking
df_genre_t.to_csv('df_genre_rankink.csv')

In [29]:
# Se crea el archivo csv de los generos
df_genre.to_csv('df_genre.csv')

In [97]:
# Se establece la funcion 
def genre(genre: str):

    ranking = df_genre_t['ranking'][df_genre_t['genres'] == genre]
    ranking = int(ranking.iloc[0])

    ptf = df_genre_t['playtime_forever'][df_genre_t['genres'] == genre]
    ptf = int(ptf.iloc[0])

    info = {'Ranking': ranking,
            'playtime': ptf}

    return info

In [98]:
genre('Indie')

{'Ranking': 2, 'playtime': 1475182802}

4. FUNCION USER FORGENRE

Creacion de la funcion

Top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

In [87]:
# Se Crea la Funcion 
def userforgenre(genero):
    data = df_genre[df_genre['genres'] == genero]
    top = data.groupby(['user_url', 'user_id'])['playtime_forever'].sum().nlargest(5).reset_index()
    
    top_users = {}
    for index, row in top.iterrows():
        user_info = {
            'user_id': row['user_id'],
            'user_url': row['user_url']
        }
        top_users[index + 1] = user_info
    
    return top_users

In [99]:
userforgenre('Indie')

{1: {'user_id': 'REBAS_AS_F-T',
  'user_url': 'http://steamcommunity.com/id/REBAS_AS_F-T'},
 2: {'user_id': 'jimmynoe',
  'user_url': 'http://steamcommunity.com/id/jimmynoe'},
 3: {'user_id': 'shinomegami',
  'user_url': 'http://steamcommunity.com/id/shinomegami'},
 4: {'user_id': 'Steamified',
  'user_url': 'http://steamcommunity.com/id/Steamified'},
 5: {'user_id': 'idonothack',
  'user_url': 'http://steamcommunity.com/id/idonothack'}}

5. FUNCION DEVELOPER

Desarrollo y creacion

Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora

In [30]:
# Se renombra la columna 'id' para unirla con el dataframe anterior y  se eliminan los duplicados

price_developer = df_game[['price', 'year', 'developer', 'id']]
price_developer = price_developer.rename(columns={'id':'item_id'})
df_developer = price_developer.drop_duplicates()
df_developer.head()

,price,year,developer,item_id
0,4.99,2018.0,Kotoshiro,761140.0
1,0.00,2018.0,Secret Level SRL,643980.0
2,0.00,2017.0,Poolians.com,670290.0
3,0.99,2017.0,彼岸领域,767400.0
4,3.99,2018.0,Trickjump Games Ltd,772540.0


In [93]:
# Se crea el Dataframe developer
df_developer.to_csv('df_developer.csv')

In [31]:
# Se obtiene la funcion desarrollador
def developer(desarrollador):
    data = df_developer[df_developer['developer'] == desarrollador]
    cantidad_anual = data.groupby('year')['item_id'].count()
    cantidad_free = data[data['price'] == 0.0].groupby('year')['item_id'].count()
    porcentaje_gratis = (cantidad_free / cantidad_anual * 100).fillna(0).astype(int)

    result = {
        'cantidad anual': cantidad_anual.to_dict(),
        'porcentaje free anual': porcentaje_gratis.to_dict()
    }
    
    return result

In [100]:
developer('Kotoshiro')

{'cantidad anual': {2018.0: 1}, 'porcentaje free anual': {2018.0: 0}}

6. FUNCION SENTIMENT ANALYSIS

Desarrollo y creacion 

Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

In [3]:
#Se extrae el año de creacion de los juegos, Se renombra la columna 'id' para unirla con el dataframe anterior 
# y se eliminan los duplicados

año_game = df_game[['id', 'year']]
año_game = año_game.rename(columns={'id':'item_id'})
año_game = año_game.drop_duplicates()
año_game.head()

,item_id,year
0,761140.0,2018.0
1,643980.0,2018.0
2,670290.0,2017.0
3,767400.0,2017.0
4,772540.0,2018.0


In [4]:
# Se une el Dataframe año_game a df_reviu
df_reviu = df_reviu.merge(año_game, on='item_id')


In [5]:
df_reviu.to_csv('df_reviu.csv')

In [6]:
#Se crea la funcion
def sentiment_analysis(anio):
    # Filtrar las reseñas del desarrollador específico
    año_reviu = df_reviu[df_reviu['year'] == anio]
    
    # Inicializar un diccionario para contar las categorías de sentimiento
    conteo_sentiment = {'Negative': 0, 'Neutral': 0, 'Positive': 0}
    
    # Iterar a través de las reseñas del desarrollador
    for index, row in año_reviu.iterrows():
        sentiment = row['sentiment_analisy']
        categoria = ''
        
        # Asignar la categoría de sentimiento correspondiente
        if sentiment == 0:
            categoria = 'Negative'
        elif sentiment == 1:
            categoria = 'Neutral'
        elif sentiment == 2:
            categoria = 'Positive'
        
        # Incrementar el contador correspondiente en el diccionario
        conteo_sentiment[categoria] += 1
    
    return conteo_sentiment

In [7]:
sentiment_analysis(2009)

{'Negative': 413, 'Neutral': 470, 'Positive': 1091}